# Introduction

This report contains a preliminary automatic analysis of MRD results for one cfDNA sample with one matched signature and potentially a set of control signatures, either from unmatched patients or elsewhere. It characterizes the different signatures in the study in terms of mutation numbers, mutation types (ref&alt bases) and allele fractions, then calculates the tumor fraction for each of the signatures. There is a set of filters applied both to the signatures and to the cfDNA reads (FeatureMap entries), results are shown both with and without those filters.

This notebook can also be used as a template for more refined analyses.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ugbio_mrd.mrd_utils as mrd

In [ ]:
from ugbio_core.plotting_utils import set_pyplot_defaults

set_pyplot_defaults()
%matplotlib inline

In [ ]:
# input parameters
features_file_parquet = None
signatures_file_parquet = None
signature_filter_query_default = "(norm_coverage <= 2.5) and (norm_coverage >= 0.6)"
signature_filter_query = signature_filter_query_default
read_filter_query_default = "qual>=60"
read_filter_query = read_filter_query_default
featuremap_df_file = None
output_dir = None
basename = None

In [ ]:
if features_file_parquet is None:
    raise ValueError("Required input features_file_parquet not provided")
if signatures_file_parquet is None:
    raise ValueError("Required input signatures_file_parquet not provided")
if featuremap_df_file is None:
    raise ValueError("Required input featuremap_df_file not provided")

In [ ]:
# read and filter df_features
df_features, df_features_filt, filtering_ratio = mrd.read_and_filter_features_parquet(
    features_file_parquet, read_filter_query
)

In [ ]:
# read and filter df_signatures
df_signatures, df_signatures_filt = mrd.read_and_filter_signatures_parquet(
    signatures_file_parquet, signature_filter_query, filtering_ratio
)

# Filters applied 

In [ ]:
filter_descriptions = {
    "ug_hcr": "In UG High Confidence Region",
    "giab_hcr": "In GIAB (HG001-007) High Confidence Region",
    "not ug_mrd_blacklist": "Not in UG MRD Blacklist",
    "not id": "Not in dbsnp",
    "af": "Allele fraction filter",
    "filtering_ratio": "Minimum ratio of read passing read filters in locus",
    "nunique == 1": "Locus only in one signature in this cohort",
    "norm_coverage": "Filtering by coverage, normalized to median",
    "X_SCORE": "Filtering by log likelihood score (effective BQ)",
    "X_EDIST": "Filtering by edit distance from the reference",
    "max_softclip_len": "Filtering by maximal softclip length",
    "X_LENGTH": "Filtering by fragment length",
    "rq": "Filtering by read quality",
    "qual": "Filtering by SNVQ",
}
print("Filters applied to signature:")
for query_string in signature_filter_query.replace("(", "").replace(")", "").split("and"):
    x = query_string.strip()
    desc = filter_descriptions.get(x.split("<")[0].split(">")[0].strip(), "<Description unavailable>")
    print(f"  - {desc}, query='{x}'")
print("\n\n")
print("Filters applied to reads:")
for filter_query in read_filter_query.replace("(", "").replace(")", "").split("and"):
    x = filter_query.strip()
    desc = filter_descriptions.get(x.split("<")[0].split(">")[0].strip(), "<Description unavailable>")
    print(f"  - {desc}, query='{x}'")

# Matched signature/s analysis

## Mutation types

In [ ]:
for plot_signature in df_signatures.query("signature_type=='matched'")["signature"].unique():
    mrd.plot_signature_mutation_types(
        df_signatures_in=df_signatures.query(f"signature == '{plot_signature}'"),
        signature_filter_query_in=signature_filter_query,
    )

Mutation type distribution before and after applying signature filters. Generally a reduction in overall number of mutations is expected, but the distribution should be mostly unchanged - significant changes could be an indication of artefacts and warrant looking into the signature data.

## Allele fractions 

In [ ]:
for plot_signature in df_signatures.query("signature_type=='matched'")["signature"].unique():
    mrd.plot_signature_allele_fractions(
        df_signatures_in=df_signatures.query(f"signature == '{plot_signature}'"),
        signature_filter_query_in=signature_filter_query,
    )

Allele fraction distribution before and after applying signature filters. Genrally a reduction in overall number of mutations is expected, and potentially a minimal allele fraction filter is applied. Allele fraction is an indication of the tumor sample purity, typical values in the range 40-50% are considered excellent, 30-40% considered good, 20-30% considered okay and below 20% considered low and might affect the validty of the results.

# Tumor fractions 

### Tumor fraction denominator
Tumor fraction = # reads intersect with signature / # all reads. <br>
To account for the query filtering, the denominator is mutiplied by the fraction of reads that pass reads filtering in the SRSNV test set. <br>
In case of filtering by QUAL (interpolation of ML_QUAL), the denominator is the fraction of reads with SNVQ > qual_threshold.


In [ ]:
denom_ratio = pd.read_parquet(featuremap_df_file).query("label").eval(read_filter_query).mean()

print(f"Denominator ratio: {denom_ratio:.2f}")

## Filtered reads, filtered signatures

In [ ]:
df_tf_filt, df_supporting_reads_per_locus_filt = mrd.get_tf_from_filtered_data(
    df_features_filt,
    df_signatures_filt,
    plot_results=True,
    title="Filtered reads and signatures",
    denom_ratio=denom_ratio,
)
display(df_tf_filt)

Tumor fractions measured for the cfDNA sample against the matched signature [red], controls signatures [blue], and database controls [green]. The boxplot shows the distribution of the background and the median value is annotated. Using the background information can aid in determining whether a detected results is statistically significant.

## Filtered reads, unfiltered signatures

In [ ]:
df_tf_unfilt, df_supporting_reads_per_locus_unfilt = mrd.get_tf_from_filtered_data(
    df_features_filt,
    df_signatures,
    plot_results=True,
    title="Filtered reads and unfiltered signatures",
    denom_ratio=denom_ratio,
)
display(df_tf_unfilt)

## Unfiltered reads, filtered signatures

In [ ]:
df_tf_unfilt2, df_supporting_reads_per_locus_unfilt2 = mrd.get_tf_from_filtered_data(
    df_features,
    df_signatures_filt,
    plot_results=True,
    title="Unfiltered reads and filtered signatures",
    denom_ratio=1,
)
display(df_tf_unfilt2)

In [ ]:
## Write tf tables to a hdf file
output_h5_file = os.path.join(output_dir, basename + ".tumor_fraction.h5")
h5_dict = {
    "df_tf_filt_signature_filt_featuremap": df_tf_filt,
    "df_tf_unfilt_signature_filt_featuremap": df_tf_unfilt,
    "df_tf_filt_signature_unfilt_featuremap": df_tf_unfilt2,
    "df_supporting_reads_per_locus_filt_signature_filt_featuremap": df_supporting_reads_per_locus_filt,
    "df_supporting_reads_per_locus_unfilt_signature_filt_featuremap": df_supporting_reads_per_locus_unfilt,
    "df_supporting_reads_per_locus_filt_signature_unfilt_featuremap": df_supporting_reads_per_locus_unfilt2,
}
for key, val in h5_dict.items():
    val.to_hdf(output_h5_file, key=key, mode="a")

# Allele fractions of plasma-matched and unmeatched variants

In [ ]:
mrd.plot_vaf_matched_unmatched(df_supporting_reads_per_locus_filt, df_signatures_filt)

# Control signature/s analysis

## Mutation types 

In [ ]:
for plot_signature in sorted(df_signatures.query("signature_type!='matched'")["signature"].unique()):
    try:
        mrd.plot_signature_mutation_types(
            df_signatures_in=df_signatures.query(f"signature == '{plot_signature}'"),
            signature_filter_query_in=signature_filter_query,
        )
    except Exception as e:
        print(f"Exception when plotting for {plot_signature}:\n{str(e)}")

## Allele fractions 

In [ ]:
for plot_signature in sorted(df_signatures.query("signature_type!='matched'")["signature"].unique()):
    try:
        mrd.plot_signature_allele_fractions(
            df_signatures_in=df_signatures.query(f"signature == '{plot_signature}'"),
            signature_filter_query_in=signature_filter_query,
        )
    except Exception as e:
        print(f"Exception when plotting for {plot_signature}:\n{str(e)}")

# cfDNA read length distributions 

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(20, 10), sharex=True)
fig.subplots_adjust(hspace=0.4, wspace=0.3)

max_value = 0
for ax, title, x in zip(
    axs.flatten(),
    [
        "Matched reads (from tumor)\nunfiltered",
        "Matched reads (from tumor)\nfiltered",
        "Unmatched reads (not tumor)\nunfiltered",
        "Unmatched reads (not tumor)\nfiltered",
    ],
    [
        df_features.query("signature_type!='matched'")["X_LENGTH"],
        df_features.query(f"signature_type!='matched' and {read_filter_query}")["X_LENGTH"],
        df_features.query("signature_type!='matched'")["X_LENGTH"],
        df_features.query(f"signature_type!='matched' and {read_filter_query}")["X_LENGTH"],
    ],
    strict=False,
):
    plt.sca(ax)
    plt.title(title, y=1.05, fontsize=28)
    max_value = max(max_value, x.max())
    x.plot.hist(bins=np.arange(0.5, max(250, max_value)))
for ax in axs[-1, :]:
    ax.set_xlabel("Read length", fontsize=32)

Distribution of read lengths for cfDNA reads, both matched and unmatched. Not all of the reads are sequenced through, so the longer reads might be limited by read rather than insert length. Differences in the distributions between matched and unmatched reads could be used for more refined filtering of reads.